In [16]:
"""
===================================================
Faces recognition example using eigenfaces and SVMs
===================================================

The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

  .. _LFW: http://vis-www.cs.umass.edu/lfw/

  original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html

"""

'\n===================================================\nFaces recognition example using eigenfaces and SVMs\n===================================================\n\nThe dataset used in this example is a preprocessed excerpt of the\n"Labeled Faces in the Wild", aka LFW_:\n\n  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)\n\n  .. _LFW: http://vis-www.cs.umass.edu/lfw/\n\n  original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html\n\n'

In [17]:
print __doc__

from time import time
import logging
import pylab as pl
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Automatically created module for IPython interactive environment


In [18]:
###############################################################################
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)

# for machine learning we use the data directly (as relative pixel
# position info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print "Total dataset size:"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_classes: %d" % n_classes

2018-02-22 12:01:40,157 Loading LFW people faces from C:\Users\MLUSER\scikit_learn_data\lfw_home


Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [24]:
###############################################################################
# Split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

###############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
components = [10, 15, 25, 50, 100, 250, 500]

for n_components in components:
    print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
    t0 = time()
    pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
    print "done in %0.3fs" % (time() - t0)

    eigenfaces = pca.components_.reshape((n_components, h, w))

    print "Projecting the input data on the eigenfaces orthonormal basis"
    t0 = time()
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    print "done in %0.3fs" % (time() - t0)


    ###############################################################################
    # Train a SVM classification model

    print "Fitting the classifier to the training set"
    t0 = time()
    param_grid = {
             'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              }
    # for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    print "done in %0.3fs" % (time() - t0)
    print "Best estimator found by grid search:"
    print clf.best_estimator_


    ###############################################################################
    # Quantitative evaluation of the model quality on the test set

    print "Predicting the people names on the testing set"
    t0 = time()
    y_pred = clf.predict(X_test_pca)
    print "done in %0.3fs" % (time() - t0)

    print classification_report(y_test, y_pred, target_names=target_names)
    print confusion_matrix(y_test, y_pred, labels=range(n_classes))

Extracting the top 10 eigenfaces from 966 faces
done in 0.123s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.027s
Fitting the classifier to the training set


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


done in 44.835s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.011s
                   precision    recall  f1-score   support

     Ariel Sharon       0.10      0.15      0.12        13
     Colin Powell       0.43      0.50      0.46        60
  Donald Rumsfeld       0.21      0.30      0.24        27
    George W Bush       0.67      0.57      0.62       146
Gerhard Schroeder       0.21      0.24      0.22        25
      Hugo Chavez       0.33      0.20      0.25        15
       Tony Blair       0.47      0.42      0.44        36

      avg / total       0.49      0.46      0.47       322

[[ 2  5  2  3  0  0  1]
 [ 8 30  6 10  3  0  3]
 [ 2  9  8  5  1  0  2]
 [ 6 20 11 83 15  3  8]
 [ 0  2  4  8  6  2  3]
 

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


done in 13.964s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.015s
                   precision    recall  f1-score   support

     Ariel Sharon       0.36      0.31      0.33        13
     Colin Powell       0.67      0.72      0.69        60
  Donald Rumsfeld       0.43      0.48      0.46        27
    George W Bush       0.75      0.77      0.76       146
Gerhard Schroeder       0.65      0.44      0.52        25
      Hugo Chavez       0.64      0.47      0.54        15
       Tony Blair       0.55      0.58      0.57        36

      avg / total       0.66      0.66      0.66       322

[[  4   3   2   4   0   0   0]
 [  5  43   1  10   1   0   0]
 [  1   4  13   9   0   0   0]
 [  1  10  12 113   2   1   

...

done in 0.283s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.052s
Fitting the classifier to the training set
done in 11.795s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.005, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.035s
                   precision    recall  f1-score   support

     Ariel Sharon       0.60      0.69      0.64        13
     Colin Powell       0.83      0.90      0.86        60
  Donald Rumsfeld       0.74      0.63      0.68        27
    George W Bush       0.88      0.94      0.91       146
Gerhard Schroeder       0.81      0.68      0.74        25
      Hugo Chavez       0.90      0.60      0.72        15
       Tony Blair       0.82      0.75      0.78        36

      avg / total       0.84      0.84      

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


done in 0.741s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.088s
Fitting the classifier to the training set
done in 29.483s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.102s
                   precision    recall  f1-score   support

     Ariel Sharon       0.56      0.77      0.65        13
     Colin Powell       0.72      0.90      0.80        60
  Donald Rumsfeld       0.90      0.70      0.79        27
    George W Bush       0.92      0.90      0.91       146
Gerhard Schroeder       0.78      0.72      0.75        25
      Hugo Chavez       0.73      0.53      0.62        15
       Tony Blair       0.90      0.78      0.84        36

      avg / total       0.84      0.83      

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


done in 1.307s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.126s
Fitting the classifier to the training set
done in 54.244s
Best estimator found by grid search:
SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting the people names on the testing set
done in 0.138s
                   precision    recall  f1-score   support

     Ariel Sharon       0.52      0.85      0.65        13
     Colin Powell       0.68      0.85      0.76        60
  Donald Rumsfeld       0.64      0.67      0.65        27
    George W Bush       0.88      0.77      0.82       146
Gerhard Schroeder       0.67      0.64      0.65        25
      Hugo Chavez       0.64      0.60      0.62        15
       Tony Blair       0.81      0.69      0.75        36

      avg / total       0.77      0.75     

In [20]:
###############################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i].reshape((h, w)), cmap=pl.cm.gray)
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                         for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

pl.show()

Variance of First Component:  0.193465266134
Variance of Second Component:  0.151168431382


In [ ]:
print 'Variance of First Component: ', pca.explained_variance_ratio_[0]
print 'Variance of Second Component: ', pca.explained_variance_ratio_[1]